In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2

# **Preparation and Pre-processing**

In [ ]:
dataset = keras.datasets.mnist

(X_raw_train, y_train), (X_raw_test, y_test) = dataset.load_data()

X_train = np.zeros((X_raw_train.shape[0], 108))
X_test = np.zeros((X_raw_test.shape[0], 108))

hog = cv2.HOGDescriptor((28, 28), (14, 14), (7, 7), (14, 14), 12)

for n in range(X_raw_train.shape[0]):
    X_train[n] = hog.compute(X_raw_train[n]).reshape(1, -1)
    
for n in range(X_raw_test.shape[0]):
    X_test[n] = hog.compute(X_raw_test[n]).reshape(1, -1)

In [ ]:
_, ax = plt.subplots(8, 8, figsize=(10, 10))
index = 0

for i in range(8):
    for j in range(8):
        cell = ax[i][j]
        cell.set_xticks([])
        cell.set_yticks([])
        cell.grid(False)
        cell.imshow(X_raw_test[index], cmap='gray')
        index += 1
        
plt.show()

Now let's calculate the prior probability p(y). This will actually be a vector containing the probabilities of occurrence for each class in our dataset. That is, the numbers from 0 to 9.

In [ ]:
_, prob_y = np.unique(y_train, return_counts=True)
prob_y = prob_y / len(y_train)
num_classes = len(prob_y)
prob_y

Calculating the Log of the Gaussian likelihood.

In [ ]:
def log_gaussian(X, mu, sigma):
    return -(np.sum(np.log(sigma)) + 0.5 * np.sum(((X - mu) / sigma) ** 2)).reshape(-1, 1)

In [ ]:
def train_naive_bayes():
    means = np.zeros((num_classes, X_train.shape[1]), dtype=np.float64)
    stdevs = np.zeros((num_classes, X_train.shape[1]), dtype=np.float64)
    
    for k, y_k in enumerate(np.unique(y_train)):
        indices = np.where(y_k == y_train)
        means[k] = np.mean(X_train[indices], axis=0)
        stdevs[k] = np.std(X_train[indices], axis=0)
        
    return means, stdevs

In [ ]:
means, stdevs = train_naive_bayes()
print('Means for class 0:\n', means[0], '\n\nStandard deviations for class 0\n', stdevs[0])

In [ ]:
def predict(x):
    return np.argmax([np.log(prob_y[k]) + log_gaussian(x.reshape(1, -1), 
                      means[k], stdevs[k]) for k in range(num_classes)])

Finding the accuracy of our model by predicting the y value for each image in our dataset.

In [ ]:
y_pred = np.zeros((X_test.shape[0]))

for n in range(X_test.shape[0]):
    y_pred[n] = predict(X_test[n])

accuracy_score(y_test, y_pred)

In [ ]:
_, ax = plt.subplots(6, 8, figsize=(10, 10))
index = 0

for i in range(6):
    for j in range(8):
        cell = ax[i][j]
        cell.set_title('Pred: %s' % predict(X_test[index]))
        cell.set_xticks([])
        cell.set_yticks([])
        cell.grid(False)
        cell.imshow(X_raw_test[index], cmap='gray')
        index += 1
        
plt.show()

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
import pickle
file = open("model_nb.pkl",'wb')
pickle.dump(model, file)

# **Convert pkl to json and load into block** 

##**BLOCK ADDING TO CHAIN**

In [ ]:
from hashlib import sha256
import json
import time


class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0

    def compute_hash(self):
        """
        A function that return the hash of the block contents.
        """
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()


class Blockchain:
    # difficulty of our PoW algorithm
    difficulty = 2

    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """
        A function to generate genesis block and appends it to
        the chain. The block has index 0, previous_hash as 0, and
        a valid hash.
        """
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    @property
    def last_block(self):
        return self.chain[-1]

    def add_block(self, block, proof):
        """
        A function that adds the block to the chain after verification.
        Verification includes:
        * Checking if the proof is valid.
        * The previous_hash referred in the block and the hash of latest block
          in the chain match.
        """
        previous_hash = self.last_block.hash

        if previous_hash != block.previous_hash:
            return False

        if not self.is_valid_proof(block, proof):
            return False

        block.hash = proof
        self.chain.append(block)
        chain_data = []
        for block in blockchain.chain:
            chain_data.append(block.__dict__)
        with open('data.json', 'w') as json_file:
            json.dump({"length": len(chain_data),
                       "chain": chain_data}, json_file)
        return True

    def is_valid_proof(self, block, block_hash):
        """
        Check if block_hash is valid hash of block and satisfies
        the difficulty criteria.
        """
        return (block_hash.startswith('0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())

    def proof_of_work(self, block):
        """
        Function that tries different values of nonce to get a hash
        that satisfies our difficulty criteria.
        """
        block.nonce = 0

        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()

        return computed_hash

    def add_new_transaction(self, transaction):
        self.unconfirmed_transactions.append(transaction)

    def mine(self):
        """
        This function serves as an interface to add the pending
        transactions to the blockchain by adding them to the block
        and figuring out Proof Of Work.
        """
        if not self.unconfirmed_transactions:
            return False

        last_block = self.last_block

        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)

        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)

        self.unconfirmed_transactions = []
        return new_block.index
blockchain = Blockchain()
tx_data = model_json
blockchain.add_new_transaction(tx_data)
blockchain.mine()

# **WRITING TIME CSV**

In [ ]:
import csv
header = 'W'
file = open('Writing_Time.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for i in range(0,50):
    start_time = time.time()
    blockchain = Blockchain()
    tx_data = model_json
    blockchain.add_new_transaction(tx_data)
    blockchain.mine()
    end_time = time.time() - start_time
    to_append = f'{end_time}'    
    file = open('Writing_Time.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())

# **READING TIME CSV**

In [ ]:
import csv
def get_chain():
    chain_data = []
    for block in blockchain.chain:
        chain_data.append(block.__dict__)
    return json.dumps({"length": len(chain_data),
                       "chain": chain_data})
header = 'R'
file = open('Reading_Time.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for i in range(0,50):
    start_time = time.time()
    get_chain()
    end_time = time.time() - start_time
    to_append = f'{end_time}'    
    file = open('Reading_Time.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())
            